# Explore SVD based word embedding

This notebook explores word embeddings that are based on SVD of a possibly transformed word-context matrix. Starting with the raw matrix `counts`

- possilby transform `counts` (e.g. log data, do PPMI transform, normalize rows)
- compute SVD of new matrix
- explore word embeddings e.g. scores plots, word similarity
- explore loadings

The material in this notebook is covered in chapters 15/16 of SLP3
- https://web.stanford.edu/~jurafsky/slp3/15.pdf
- https://web.stanford.edu/~jurafsky/slp3/16.pdf


## data

The github repo comes with a small data set pre computed (from a random sample of 1000 court cases). You can download larger data files from https://drive.google.com/open?id=0B40b05f-8LWtVGsybWw4OTVyV00 then place them in the data/ folder.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import heapq

from scipy.sparse.linalg import svds, norm
from scipy.sparse import diags, csr_matrix, dok_matrix

# import local code files
import sys, os
sys.path.append(os.getcwd() + '/code/')

# working with word embeddings
from save import load_vocabulary, load_matrix
from ppmi import calc_ppmi
from word_similarity import similarity, closest, vec, angle_between, word_angles
from transform_counts import remove_zero_count_words, normalize_rows


# exploring/visualizing scores/loadings
from scores_viz import scores_plot, filter_scores
from explore_loadings import top_loading_components, top_loading_words, top_loading_words_df

# only import this if you have plot.ly installed
# from viz_plotly import interactive_scores_plot

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
w2i, i2w = load_vocabulary('data/vocab_small_ex.txt')
co_counts = load_matrix('data/co_counts_small_ex')
word_counts = np.load('data/word_counts_small_ex.npy')

# # uncomment this code if you have the larger data file
# w2i, i2w = load_vocabulary('data_no_github/vocab_10000.txt')
# co_counts = load_matrix('data_no_github/co_counts_10000')
# word_counts = np.load('data_no_github/word_counts_10000.npy')

In [3]:
co_counts

<39424x39424 sparse matrix of type '<class 'numpy.float64'>'
	with 11250370 stored elements in Compressed Sparse Row format>

# transform counts

In [4]:
# normalized row by row norms
co_counts_normed = diags(1.0/norm(co_counts, axis=1)) * co_counts 

In [5]:
co_counts_log = co_counts.copy()
co_counts_log.data = np.log(1 + co_counts.data)

In [6]:
ppmi = calc_ppmi(co_counts)

# SVD of counts matrix

If $X \in \mathbb{R}^{n \times d}$ is a data matrix with the n observations on rows then the rank K SVD is given by
$$X \approx U D V^T$$
where $U \in \mathbb{R}^{n \times K}$ is the matrix of normalized scores and  $V \in \mathbb{R}^{d \times K}$ is the matrix of loadings.

If X is a word-context matrix then U (or UD) gives a word embedding.

In [8]:
%%time
k = 10

normed_scores, sing_vals, loadings = svds(co_counts_normed, k)

loadings = loadings.T
un_normed_scores = normed_scores * sing_vals

CPU times: user 10.9 s, sys: 194 ms, total: 11.1 s
Wall time: 2.83 s


In [ ]:
# set the word embedding that is used in the rest of the script
embedding = un_normed_scores

# Explore word embedding (scores)

In [ ]:
scores_plot(embedding,
            start=1,
            n_comp=5,
            title='',
            comp_names=None)

# simlarity

In [ ]:
word1 = 'lawyer'
word2 = 'lawyers'

similarity(word1, word2, embedding, w2i, sim='angle')

In [ ]:
word = 'lawyers'
closest(word, embedding, w2i, N=50)

In [ ]:
angles = word_angles(word, embedding, w2i)
plt.hist(angles, bins=100)
plt.xlabel('angles')
plt.title('angles between %s and all other words' % word)

# explore loadings

In [ ]:
top_loading_df = top_loading_words_df(loadings, i2w, n=100)

top_loading_df

In [ ]:
comp_numer = 100  # which loading
n = 100 # how many components to show

plt.figure(figsize=[40, 20])

plt.subplot(1,2,1)
top_loading_components(loadings[:, comp_numer], i2w, n, comp_numer)


plt.subplot(1,2,2)
plt.scatter(range(loadings.shape[0]), loadings[:, comp_numer], color='grey')